In [1]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from ces_fun_util import *
#from fun_utils_eq import *

In [2]:
## parameter values
alpha = 0.85           # labor share parameter in manufacturing
#alpha = 0.2
theta = 4             # scopevec for comparative advantage
sigma = 1      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1  # elasticity of demand for each individual manufactured good j at Foreign
epsilonD = alpha + (1 - alpha) * sigma  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar  #Foreign's elasticity of demand for embodied energy
#beta = 2.274853
#gamma= 0.784877595
beta=1.892412
gamma=0.807998928
epsilonS = 0.5  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5  #Foreign's energy supply elasticity: betastar/(1 - betastar)

In [3]:
## scenario switch
case=3 # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0 # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [4]:
paralist = (theta, sigma, sigmastar, epsilonS,epsilonSstar, beta, gamma, logit)

In [5]:
class taxrow_eq:
    def __init__(self, pe, te, tb_mat, df, tax_scenario, varphi, paralist):
        self.df = df
        self.tax_scenario = tax_scenario
        self.varphi = varphi
        self.paralist = paralist
        self.prop = 1
        self.te = te
        self.pe = pe
        self.tb_mat = tb_mat

    ## computes equilibrium price and taxes. Also computes other equilibrium values (ie consumption, production, value of exports/imports)
    ## and stores them in self. 
    def opt_tax(self):
        tax_scenario = self.tax_scenario
        varphi = self.varphi
        df = self.df
        te = varphi
        self.region = df['region_scenario']
        pes = np.append([self.pe], np.arange(0.1, 2, 0.2))
        tbs = np.append([self.tb_mat[0]], np.arange(0,1.5,0.2))
        props = np.append([self.tb_mat[1]], np.arange(0,1.1,0.5))
        self.conv = 1
        if (tax_scenario['tax_sce'] == 'global'):
            tbs = [0]
        
        if tax_scenario['tax_sce'] != 'purete' and tax_scenario['tax_sce'] != 'EP_hybrid':
            for prop in props:
                for tb in tbs:
                    for pe in pes:
                        res = minimize(self.minuswelfare, [pe, tb, prop], bounds=[(0.01, np.inf), (0, np.inf), (0, 1)],
                               method='nelder-mead', args=(te), tol=0.000001, options={'maxfev': 100000})
                        if res.fun <= 0.0001:
                            opt_val = res.x
                            break
                    else:
                        continue
                    break
                else: 
                    continue
                break
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            opt_val = res.x
            #print(res.fun, df['region_scenario'], tax_scenario['tax_sce'], varphi)
            
        pes = np.append([self.pe], np.arange(0.1, 2, 0.5))
        props = np.append([self.tb_mat[1]], np.arange(0,2,0.5))
        if tax_scenario['tax_sce'] == 'EP_hybrid':
            for pe in pes:
                for tb in tbs:
                    for prop in props:
                        res = minimize(self.minuswelfare, [pe, tb, prop], bounds=[(0.01, np.inf), (0, np.inf), (0, np.inf)],
                               method='nelder-mead', args=(te), tol=0.000001, options={'maxfev': 100000})
                        if res.fun <= 0.0001:
                            opt_val = res.x
                            break
                    else:
                        continue
                    break
                else: 
                    continue
                break
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            opt_val = res.x
            
            
        ## not implemented yet
        if tax_scenario['tax_sce'] == 'Baseline':
            self.tb = 0
            self.prop = 1
            self.te = 0
        
        elif tax_scenario['tax_sce'] == 'global':
            self.tb = 0
            self.te = varphi

        elif tax_scenario['tax_sce'] == 'PC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]
            self.te = self.tb

        elif tax_scenario['tax_sce'] == 'EPC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]
            self.te = varphi

        elif tax_scenario['tax_sce'] == 'EP_hybrid':
            self.tb = opt_val[1]
            self.te = opt_val[2]
            self.prop = self.te #- self.tb

        elif tax_scenario['tax_sce'] == 'Unilateral' or tax_scenario['tax_sce'] == 'puretc' or tax_scenario[
            'tax_sce'] == 'puretp' or tax_scenario['tax_sce'] == 'EC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]

            if tax_scenario['tax_sce'] == 'puretc' or tax_scenario['tax_sce'] == 'puretp':
                self.te = self.tb

            elif tax_scenario['tax_sce'] == 'Unilateral' or tax_scenario['tax_sce'] == 'EC_hybrid':
                self.te = self.varphi

        elif tax_scenario['tax_sce'] == 'purete':
            res = minimize(self.minuswelfare, [self.pe, self.te], bounds=[(0.001, np.inf), (0, np.inf)], method='nelder-mead', args=(0),
                           tol=0.000001, options={'maxfev': 100000})
            
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            #print(res.fun)
            self.te = res.x[1]
            tb_mat = [0, 1]
            self.tb = tb_mat[0]
            self.prop = tb_mat[1]
        #print(tax_scenario['tax_sce'], res)

    def minuswelfare(self, p, te):
        pe = p[0]
        tb_mat = p[1:]
        varphi = self.varphi
        tax_scenario = self.tax_scenario
        paralist = self.paralist
        df = self.df

        if tax_scenario['tax_sce'] == 'purete':
            te = p[1]
            tb_mat = [0, 1]

        obj = self.comp_obj(pe, te, tb_mat, varphi, tax_scenario, paralist, df)

        return obj

    ## compute the objective value, currently the objective is to minimize negative welfare
    ## also saves optimal results in self.
    def comp_obj(self, pe, te, tb_mat, varphi, tax_scenario, paralist, df):

        theta, sigma, sigmastar, epsilonS, epsilonSstar, beta, gamma, logit = paralist
        ## compute extraction tax, and jbar's
        te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = computejbar(paralist, pe, te, varphi, tb_mat, tax_scenario, df)

        jxbar_prime = jxbar_hat * df['jxbar']
        jmbar_prime = jmbar_hat * df['jmbar']
        j0_prime = j0_hat * df['jxbar']

        jvals = (j0_hat, j0_prime, jxbar_hat, jxbar_prime, jmbar_hat, jmbar_prime)

        # if te is too large, Home stop producing
        petbte = pe + tb_mat[0] - te
        z = pe + tb_mat[0] >= te
        petbte = petbte * z

        Qe_prime, Qestar_prime = compute_qe(tax_scenario, petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)

        Cey_prime, Cex1_prime, Cex2_prime, Cex_prime, Cem_prime, CeFF_prime, Cex_hat, Cex1_hat, Cem_hat = comp_ce(
            pe, tb_mat, jvals, paralist, df, tax_scenario)

        Ge_prime = Cey_prime + Cex_prime
        Gestar_prime = CeFF_prime + Cem_prime
        Ce_prime = Cey_prime + Cem_prime
        Cestar_prime = CeFF_prime + Cex_prime
        Qeworld_prime = Qe_prime + Qestar_prime

        consvals = (Cey_prime, Cex1_prime, Cex2_prime, Cex_prime, 
                    Cem_prime, CeFF_prime, Cex_hat, Cex1_hat, Cem_hat,
                    Ce_prime, Cestar_prime)

        Vgy, Vgystar, Vgx1_prime, Vgx2_prime, Vgx_prime, Vgm_prime, Vgx, Vgm = comp_vg(pe, tb_mat, jvals, consvals,
                                                                                           df, tax_scenario, paralist)

        pai_g = Vgx - (pe + tb_mat[0]) * df['CeFH'] / (1 - alpha)
        subsidy_ratio = 1- ((1-jxbar_prime) * j0_prime / ((1-j0_prime) * jxbar_prime))**(1/theta)

        ## compute Ve values
        Ve_prime, Vestar_prime = comp_ve(pe, tb_mat, consvals, tax_scenario)

        ## compute more Vg values
        vgvals = Vgx_prime, Vgm_prime
        Vg, Vg_prime, Vgstar, Vgstar_prime = comp_vgfin(pe, tb_mat, consvals, vgvals, jvals, paralist, df, tax_scenario)

        Lg, Lg_prime, Lgstar, Lgstar_prime = comp_lg(pe, tb_mat, Ge_prime, Gestar_prime, consvals, paralist, df, tax_scenario)

        leakage1, leakage2, leakage3 = comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df)

        lgvals = Lg, Lgstar, Lg_prime, Lgstar_prime
        vgvals = Vg, Vgstar, Vg_prime, Vgstar_prime
        delta_Le, delta_Lestar, delta_U, delta_Vg, delta_Vgstar = comp_delta(lgvals, vgvals, Qeworld_prime, df, jvals, pe, petbte, tb_mat, tax_scenario, varphi, paralist)

        chg_extraction, chg_production, chg_consumption, chg_Qeworld = comp_chg(df, Qestar_prime, Gestar_prime,
                                                                                Cestar_prime, Qeworld_prime)

        welfare = delta_U / Vg * 100
        welfare_noexternality = (delta_U + varphi * (Qeworld_prime - df['Qeworld'])) / Vg * 100
        
        self.results = assign_val(Ge_prime, Gestar_prime, Lg_prime, Lgstar_prime, Qe_prime, Qestar_prime, Qeworld_prime,
                                  Ve_prime, Vestar_prime, Vgx1_prime, Vgx2_prime, Vgx_prime, Vgm_prime, Vg_prime,
                                  Vgstar_prime, chg_Qeworld, chg_consumption, chg_extraction, chg_production, delta_Le, 
                                  delta_Lestar, leakage1, leakage2, leakage3,
                                  pai_g, pe, subsidy_ratio, varphi, welfare, welfare_noexternality, jvals, consvals, delta_Vg, delta_Vgstar)

        diff, diff1, diff2 = comp_diff(consvals, jvals, Ge_prime, Gestar_prime, Qe_prime, Qestar_prime, Qeworld_prime,
                                       Vgx2_prime, pe, tax_scenario, tb_mat, te, varphi, paralist, df)

        obj_val = abs(diff1) + abs(diff) + abs(diff2)
        return obj_val

    ## retrive the pandas series object containing equilibrium values (prices, taxes, consumption etc)
    def retrive(self):
        ret = self.results
        ret['tb'] = self.tb
        ret['prop'] = self.prop
        ret['te'] = self.te
        ret['region_scenario'] = self.region
        ret['conv'] = self.conv
        
        return ret

In [6]:
## import BAU values (seven regional scenarios in the order of US, EU, OECD, World, China, OECD plus China)
if case==2:
    df = pd.read_csv("../../output/BaselineCarbon_2015_noTradeinGoods.csv",index_col=['region_scenario','regionbase'],header='infer')
elif case==3:
    df = pd.read_csv("../../output/BaselineCarbon_2015.csv", index_col=['regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF'])
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF'])

## choose which regional scenario to run (runs all if not executed)
#df=df.drop([2,3,4,5,6,7])  
df = df[df['region_scenario'] == 1]
#df = df[df['region_scenario'] != 4]
df

,region_scenario,Qe,Qestar,Qeworld,CeHH,CeHF,CeFH,CeFF,Ce,Cestar,Ge,Gestar,Ceworld,Geworld,jxbar,jmbar
regionbase,,,,,,,,,,,,,,,,
US as Home,1,4.48004,27.79594,32.27598,4.5984,1.19611,0.4216,26.05987,5.79451,26.48147,5.02,27.25598,32.27598,32.27598,0.015921,0.793579


In [7]:
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','global','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1,1]},index=[1,2, 3, 4, 5, 6, 7, 8,9])
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1]},index=[1,2, 3, 4, 5, 6, 7, 8])
#tax_scenario= pd.DataFrame({'tax_sce': ['PC_hybrid','EPC_hybrid'], 'Base':[1,1]},index=[1,2])
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral'], 'Base':[0]},index=[1])
#df = pd.DataFrame({'Qe': 4.4800448, 'Qestar': 27.795946, 'Qeworld': 32.27599, 'CeHH': 4.598403, 'CeHF': 1.196111,'CeFH': 0.42160103,'CeFF': 26.059874,'Ce': 5.7945137,'Cestar': 26.481474,'Ge': 5.0200038,'Gestar': 27.255985,'Ceworld': 32.27599,'Geworld': 32.27599,'jxbar': 0.015920602214279298,'jmbar':  0.7935787194577492}, index = [1])

In [8]:
def temp(tax_scenario, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    paralist = (theta, sigma, sigmastar, epsilonS,epsilonSstar, beta, gamma, logit)
    tb_mat = [0, 1];  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)
    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    #varphilist = np.arange(0,3,0.1)
    varphilist = np.arange (0,1,0.1)
    #varphilist = [2.9]
    output=[]
    prevtb = [1,1,1,1]
    for varphi in varphilist:    
        tax_df=df.apply(mintemp, axis=1, raw=False, args=(prevtb, varphi, tax_scenario, paralist))
        prevtb = tax_df[['region_scenario','tb', 'pe', 'te', 'prop']]
        output.append(tax_df)
        print(varphi)
    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid':
        output.te[output.Qe_prime==0]=output.pe+output.tb
    print(tax_scenario['tax_sce'])
    return output

In [12]:
def mintemp(df, prevtb, varphi, tax_scenario, paralist):
    pe = 1
    te = 0.5
    tb_mat = [0,0.5]
    #print(prevtb['region_scenario'])
    
    if (type(prevtb) != list):
        curr_region = prevtb[prevtb['region_scenario'] == df['region_scenario']]
        vals = curr_region.values
        #print(vals)
        tb_mat = [vals[0][1], vals[0][4]]
        pe = vals[0][2]
        te = vals[0][3]
    tax_temp = taxrow_eq(pe, te, tb_mat, df, tax_scenario, varphi, paralist)
    tax_temp.opt_tax()
    ret = tax_temp.retrive()
    #print(ret)
    return ret

In [13]:
temp_df = tax_scenario.apply(temp, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))
#temp_df.head()

C:\Users\Alex\Desktop\Tobin\sam\Optimal-Unilateral-Carbon-Policy\Data\codes\ces\ces_fun_util.py:443: RuntimeWarning: divide by zero encountered in double_scalars
  leakage1 = -(Qestar_prime - df['Qestar']) / (Qeworld_prime - df['Qeworld'])
C:\Users\Alex\Desktop\Tobin\sam\Optimal-Unilateral-Carbon-Policy\Data\codes\ces\ces_fun_util.py:444: RuntimeWarning: divide by zero encountered in double_scalars
  leakage2 = -(Gestar_prime - df['Gestar']) / (Qeworld_prime - df['Qeworld'])
C:\Users\Alex\Desktop\Tobin\sam\Optimal-Unilateral-Carbon-Policy\Data\codes\ces\ces_fun_util.py:445: RuntimeWarning: divide by zero encountered in double_scalars
  leakage3 = -(Cestar_prime - df['Cestar']) / (Qeworld_prime - df['Qeworld'])


0.0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0.9
Unilateral


In [14]:
output_list=[]
for i in range(1,len(tax_scenario)+1):
    output_list.append(temp_df.loc[i])
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)
Outcomes

,tax_sce,regionbase,varphi,pe,tb,prop,te,jxbar_prime,jmbar_prime,j0_prime,...,chg_consumption,chg_Qeworld,pai_g,subsidy_ratio,delta_Vg,delta_Vgstar,welfare,welfare_noexternality,region_scenario,conv
0,Unilateral,US as Home,0.0,1.000000,0.000000,0.500000,0.0,0.015921,0.793579,0.015921,...,0.000000,7.105427e-15,0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,1.0,1.0
1,Unilateral,US as Home,0.1,0.998753,0.034739,0.728684,0.1,0.015925,0.793579,0.015602,...,0.018896,-1.688817e-01,-0.094133,0.005190,-0.190888,0.018777,0.021881,-0.021836,1.0,1.0
2,Unilateral,US as Home,0.2,0.997896,0.069419,0.117013,0.2,0.015939,0.793579,0.015301,...,0.028407,-3.370481e-01,-0.189199,0.010327,-0.377487,0.027978,0.087377,-0.087123,1.0,1.0
3,Unilateral,US as Home,0.3,0.997416,0.104079,0.035884,0.3,0.015961,0.793579,0.015014,...,0.028751,-5.051688e-01,-0.285267,0.015411,-0.560142,0.027880,0.196376,-0.195936,1.0,1.0
4,Unilateral,US as Home,0.4,0.997304,0.138757,0.026083,0.4,0.015991,0.793579,0.014741,...,0.020044,-6.739355e-01,-0.382420,0.020443,-0.739185,0.018640,0.348969,-0.348866,1.0,1.0
5,Unilateral,US as Home,0.5,0.997557,0.173491,0.020161,0.5,0.016028,0.793579,0.014481,...,0.002286,-8.440800e-01,-0.480758,0.025424,-0.914945,0.000280,0.545410,-0.547107,1.0,1.0
6,Unilateral,US as Home,0.6,0.998178,0.208319,0.016714,0.6,0.016071,0.793579,0.014233,...,-0.024650,-1.016406e+00,-0.580395,0.030355,-1.087753,-0.027317,0.786168,-0.792508,1.0,1.0
7,Unilateral,US as Home,0.7,0.999179,0.243282,0.015266,0.7,0.016121,0.793579,0.013997,...,-0.061051,-1.191823e+00,-0.681475,0.035235,-1.257950,-0.064444,1.071908,-1.087748,1.0,1.0
8,Unilateral,US as Home,0.8,1.000576,0.278423,0.014580,0.8,0.016177,0.793579,0.013770,...,-0.107397,-1.371405e+00,-0.784174,0.040067,-1.425901,-0.111604,1.403572,-1.436507,1.0,1.0
9,Unilateral,US as Home,0.9,1.002402,0.313793,0.013776,0.9,0.016239,0.793579,0.013553,...,-0.164430,-1.556468e+00,-0.888717,0.044850,-1.592010,-0.169581,1.782400,-1.843845,1.0,1.0


In [12]:
Outcomes[['pe','tb', 'Cey_prime', 'Cex_prime', 'Cem_prime', 'Ceystar_prime', 'Vgx_prime', 'Vgm_prime']]

,pe,tb,Cey_prime,Cex_prime,Cem_prime,Ceystar_prime,Vgx_prime,Vgm_prime
0,1.000000,0.000000,4.598400,0.421600,1.196110,26.059870,2.810667,7.974067
6,0.998648,0.042625,4.466978,0.409705,1.161925,26.085030,2.811499,7.974067
12,0.997713,0.085194,4.343070,0.398774,1.129695,26.102182,2.813905,7.974067
18,0.997174,0.127742,4.225981,0.388705,1.099239,26.111706,2.817762,7.974067
24,0.997015,0.170303,4.115094,0.379404,1.070395,26.113904,2.822960,7.974067
...,...,...,...,...,...,...,...,...
575,0.948055,4.195562,2.339016,0.318217,0.614277,22.881716,3.289357,13.531800
581,0.947481,4.237976,2.324949,0.317441,0.610582,22.892339,3.284841,13.531800
587,0.946916,4.280379,2.311076,0.316677,0.606939,22.902817,3.280394,13.531800
593,0.946359,4.322770,2.297395,0.315926,0.603346,22.913152,3.276016,13.531800


In [15]:
#outcome_t = Outcomes[Outcomes['regionbase'] == 'US as Home']
Outcomes[['pe','tb', 'Qe_prime', 'Qestar_prime', 'Ce_prime', 'Cestar_prime','Lg_prime','Lgstar_prime', 'Vg_prime', 'Vgstar_prime', 'Vgx2_prime', 'delta_Le','delta_Lestar','welfare','welfare_noexternality']]

,pe,tb,Qe_prime,Qestar_prime,Ce_prime,Cestar_prime,Lg_prime,Lgstar_prime,Vg_prime,Vgstar_prime,Vgx2_prime,delta_Le,delta_Lestar,welfare,welfare_noexternality
0,1.000000,0.000000,4.480040,27.795940,5.794510,26.481470,28.446667,154.450553,38.630067,176.543133,0.000000,0.000000,0.000000,-0.000000,0.000000
1,0.998753,0.034739,4.328497,27.778602,5.606732,26.500366,28.447500,154.449847,38.630067,176.543133,0.057013,-0.146475,-0.017328,0.021881,-0.021836
2,0.997896,0.069419,4.172249,27.766683,5.429056,26.509877,28.449925,154.447797,38.630067,176.543133,0.112697,-0.287129,-0.029226,0.087377,-0.087123
3,0.997416,0.104079,4.010810,27.760001,5.260590,26.510221,28.453839,154.444502,38.630067,176.543133,0.167177,-0.421799,-0.035893,0.196376,-0.195936
4,0.997304,0.138757,3.843604,27.758441,5.100530,26.501514,28.459151,154.440045,38.630067,176.543133,0.220564,-0.550305,-0.037449,0.348969,-0.348866
5,0.997557,0.173491,3.669936,27.761964,4.948142,26.483756,28.465781,154.434503,38.630067,176.543133,0.272955,-0.672446,-0.033935,0.545410,-0.547107
6,0.998178,0.208319,3.488963,27.770611,4.802754,26.456820,28.473657,154.427943,38.630067,176.543133,0.324441,-0.787998,-0.025306,0.786168,-0.792508
7,0.999179,0.243282,3.299635,27.784522,4.663739,26.420419,28.482716,154.420427,38.630067,176.543133,0.375103,-0.896706,-0.011413,1.071908,-1.087748
8,1.000576,0.278423,3.100626,27.803949,4.530503,26.374073,28.492899,154.412008,38.630067,176.543133,0.425011,-0.998281,0.008011,1.403572,-1.436507
9,1.002402,0.313793,2.890214,27.829298,4.402473,26.317040,28.504155,154.402737,38.630067,176.543133,0.474233,-1.092383,0.033398,1.782400,-1.843845


In [14]:
#Outcomes.to_csv('../../output_sig0/ces-05.csv'.format(theta, case), header=True)